# Topological K-means

#### Importing libraries

In [1]:
import numpy as np # Linear algebra
import pandas as pd # Data processing
import matplotlib.pyplot as plt # Data visualization
import seaborn as sns # Data visualization
import networkx as nx # Graph analysis and manipulation

from numpy import dot
from numpy import trace
from numpy.linalg import inv
import scipy.sparse._csr
%matplotlib inline

import sklearn.datasets as skdata
import sklearn.neighbors as sknn
import sklearn.utils.graph as sksp
from sklearn import preprocessing
from sklearn.cluster import KMeans
from sklearn.metrics.cluster import rand_score
from sklearn.metrics.cluster import silhouette_score

import sys
import time
import warnings
warnings.simplefilter(action='ignore')

#### Importing dataset

In [101]:
### Scikit-learn datasets
# (>) Melhor (=) Próximos (<) Pior

X = skdata.load_iris()                                                  # (150, 4)      - (<)
#X = skdata.fetch_openml(name='segment', version=2)                      # (2310, 19)    - (<)
#X = skdata.fetch_openml(name='amazon-commerce-reviews', version=1)      # (1500, 10000) - (<)
#X = skdata.fetch_openml(name='semeion', version=1)                      # (1593, 256)   - (<)
#X = skdata.fetch_openml(name='mfeat-pixel', version=3)                  # (2000, 240)   - (<)
#X = skdata.fetch_openml(name='micro-mass', version=1)                   # (360, 1300)   - (<)
#X = skdata.fetch_openml(name='monks-problems-1', version=1)             # (556, 6)      - (<)
#X = skdata.fetch_openml(name='breast-tissue', version=1)                # (106, 9)      - (<)
#X = skdata.fetch_openml(name='fri_c2_100_10', version=2)                # (100, 10)     - (<)
#X = skdata.fetch_openml(name='datatrieve', version=1)                   # (130, 8)      - (<)
#X = skdata.fetch_openml(name='fri_c3_250_25', version=2)                # (250, 25)     - (<)
#X = skdata.fetch_openml(name='cnae-9', version=2)                       # (240, 856)    - Bem pior (<)
#X = skdata.fetch_openml(name='oh5.wc', version=1)                       # (918, 3012)   - Bem pior (<)
#X = skdata.fetch_openml(name='GCM', version=1)                          # (190, 16063)  - (=)
#X = skdata.fetch_openml(name='collins', version=2)                      # (500, 22)     - (=)
#X = skdata.fetch_openml(name='pyrim', version=2)                        # (74, 27)      - (=)
#X = skdata.fetch_openml(name='balance-scale', version=1)                # (625, 4)      - (=)
#X = skdata.fetch_openml(name='tr45.wc', version=1)                      # (690, 8261)   - (>)
#X = skdata.fetch_openml(name='cloud', version=2)                        # (108, 7)      - (>)
#X = skdata.fetch_openml(name='servo', version=1)                        # (167, 4)      - (>)
#X = skdata.fetch_openml(name='AP_Breast_Lung', version=1)               # (470, 10935)  - Muito bom! (>)
#X = skdata.fetch_openml(name='leukemia', version=1)                     # (72, 7129)    - Excelente! (>)
#X = skdata.fetch_openml(name='AP_Colon_Prostate', version=1)            # (355, 10935)  - Excelente! (>)
#X = skdata.fetch_openml(name='AP_Colon_Kidney', version=1)              # (546, 10935)  - Excelente! (>)
#X = skdata.fetch_openml(name='penguins', version=1)                     # (344, 6)      - NaNs

### Lentos
#X = skdata.load_digits()                                                # (1797, 64)    - (<)
#X = skdata.fetch_openml(name='phoneme', version=1)                      # (5404, 5)     - (<)
#X = skdata.fetch_openml(name='Bioresponse', version=1)                  # (3751, 1776)  -
#X = skdata.fetch_openml(name='optdigits', version=1)                    # (5620, 64)    -
#X = skdata.fetch_openml(name='satimage', version=1)                     # (6430, 36)    -

y = X['target']
X = X['data']
k = len(np.unique(y)) # Number of unique labels
n, m = X.shape # Number of observations and variables

if type(X) == scipy.sparse._csr.csr_matrix:
    X = X.todense()
    X = np.asarray(X)
else:
    if not isinstance(X, np.ndarray):
        cat_cols = X.select_dtypes(['category']).columns
        X[cat_cols] = X[cat_cols].apply(lambda x: x.cat.codes)
        X = X.to_numpy()
        y = y.to_numpy()

#### Exploratory Data Analysis

In [20]:
df = pd.DataFrame(data=X)
df.shape # (observations, variables)

(355, 10935)

In [5]:
df.head()

,0,1,2,3,4,5,6,7,8,9,...,16053,16054,16055,16056,16057,16058,16059,16060,16061,16062
0,-73.0,-69.0,-48.0,13.0,-86.0,-147.0,-65.0,-71.0,-32.0,100.0,...,-134.0,352.0,-67.0,121.0,-5.0,-11.0,-21.0,-41.0,-967.0,-120.0
1,-16.0,-63.0,-97.0,-42.0,-91.0,-164.0,-53.0,-77.0,-17.0,122.0,...,-51.0,244.0,-15.0,119.0,-32.0,4.0,-14.0,-28.0,-205.0,-31.0
2,4.0,-45.0,-112.0,-25.0,-85.0,-127.0,56.0,-110.0,81.0,41.0,...,14.0,163.0,-14.0,7.0,15.0,-8.0,-104.0,-36.0,-245.0,34.0
3,-31.0,-110.0,-20.0,-50.0,-115.0,-113.0,-17.0,-40.0,-17.0,80.0,...,26.0,625.0,18.0,59.0,-10.0,32.0,-2.0,10.0,-495.0,-37.0
4,-33.0,-39.0,-45.0,14.0,-56.0,-106.0,73.0,-34.0,18.0,64.0,...,-69.0,398.0,38.0,215.0,-2.0,44.0,3.0,68.0,-293.0,-34.0


In [21]:
print(np.unique(y))
print('Number of unique labels: {}'.format(k))

['Colon' 'Prostate']
Number of unique labels: 2


#### Converting labels into integers

In [102]:
labels = list(np.unique(y))
numbers = np.zeros(n)
for i in range(n):
    numbers[i] = labels.index(y[i])

#### Topological K-means implementation

In [5]:
def centroid_distances(G, coord_centroids):
    distances = []
    for i in coord_centroids:
        D = nx.single_source_dijkstra(G,i)[0]
        temp = np.ones(n) * np.inf
        for j in D.keys():
            temp[j] = D[j]
        distances.append(temp)

    return np.array(distances)

def TopologicalKMeans(X):
    n_sqrt = round(np.sqrt(n))
    
    coord_centroids = np.random.choice(n, size=k, replace=False)
    centroids = X[coord_centroids] # Randomly chosen observations as centroids

    iteration = 0
    while iteration < 10:
        iteration += 1
        
        knnGraph = sknn.kneighbors_graph(X, n_neighbors=n_sqrt, mode='distance') # KNN graph
        W = knnGraph.toarray() # Adjacency matrix
        
        G = nx.from_numpy_array(W) # NetworkX graph
        distances = centroid_distances(G,coord_centroids) # Geodesic distances from each centroid to all observations

        geo_labels = np.zeros(n) # Array for labels
        for j in range(n):
            geo_labels[j] = distances[:, j].argmin() # Updating labels
        
        new_centroids = np.zeros((k, m))
        for r in range(k):
            indices = np.where(geo_labels==r)[0]
            
            if len(indices) > 0:
                sample = X[indices]
                new_centroids[r, :] = sample.mean(axis=0)
            else:
                new_centroids[r, :] = centroids[r, :]
        
        if (np.linalg.norm(centroids - new_centroids) < 0.05):
            break

        centroids = new_centroids.copy() # Updated centroids
    
    return geo_labels

#### Comparação dos algoritmos

In [109]:
def kmeans_exec(X, iterations:int, mode='classical', results=False):
    if mode not in ['classical','topological']:
        raise ValueError("Mode argument must be 'classical' or 'topological'.")

    rand_list, sc_list = [], []
    if mode == 'classical':
        start_time = time.time()
        for i in range(1, iterations+1):
            if (i == 1) or (i % 5 == 0):
                print('Iteration: {}/{}'.format(i,iterations))
            kmeans = KMeans(n_clusters=k, init='random', n_init=1).fit(X)
            rand_list.append(rand_score(y, kmeans.labels_))
            sc_list.append(silhouette_score(X, kmeans.labels_))
        end_time = time.time()

        print()
        print('Classical K-means')
        print('Elapsed time: %.4f' %(end_time - start_time))
        print()
        print('Average Rand index: %.4f' %(np.mean(rand_list)))
        print('Average Silhouette coefficient: %.4f' %(np.mean(sc_list)))

        if results == True:
            return((np.mean(rand_list),np.mean(sc_list),end_time - start_time))
    else:
        start_time = time.time()
        for i in range(1, iterations+1):
            if (i == 1) or (i % 5 == 0):
                print('Iteration: {}/{}'.format(i,iterations))
            labels = TopologicalKMeans(X)
            rand_list.append(rand_score(y, labels))
            sc_list.append(silhouette_score(X, labels))
        end_time = time.time()

        print()
        print('Topological K-means')
        print('Elapsed time: %.4f' %(end_time - start_time))
        print()
        print('Average Rand index: %.4f' %(np.mean(rand_list)))
        print('Average Silhouette coefficient: %.4f' %(np.mean(sc_list)))
        if results == True:
            return((np.mean(rand_list),np.mean(sc_list),end_time - start_time))

##### Classical K-means

In [104]:
kmeans_exec(X,30,version='classical')

Iteration: 1/20
Iteration: 5/20
Iteration: 10/20
Iteration: 15/20
Iteration: 20/20

Classical K-means
Elapsed time: 7.8340

Average Rand index: 0.5808
Average Silhouette coefficient: 0.2372


##### Topological K-means

In [105]:
kmeans_exec(X,30,version='topological')

Iteration: 1/20
Iteration: 5/20
Iteration: 10/20
Iteration: 15/20
Iteration: 20/20

Topological K-means
Elapsed time: 65.8901

Average Rand index: 0.5477
Average Silhouette coefficient: 0.1843
